# Cartola Matches
Esse notebook tem como objetivo popular uma base de dados com todas as partidas ocorridas no ano corrente do Campeonato Brasileiro série A.

### Dependências
Instalação de dependências necessárias

In [ ]:
%pip install requests
%pip install psycopg2-binary
%pip install sqlalchemy

### Bibliotecas necessárias 
Importação das bibliotecas necessárias para a execução do notebook

In [ ]:
import timeit
import requests
import json
import pandas as pd
from sqlalchemy import create_engine

### Início
Busca dos dados na API do Cartola

In [ ]:
start = timeit.default_timer()

In [ ]:
home_page = requests.get('https://api.cartolafc.globo.com/partidas')
home_json = json.loads(home_page.content)
last_turn = home_json['rodada']
print(f'Rodada atual: {last_turn}')

In [ ]:
matches = []
for turn in range(1, last_turn):
    turn_page = requests.get(f'https://api.cartolafc.globo.com/partidas/{turn}')
    turn_json = json.loads(turn_page.content)
    turn_matches = turn_json['partidas']
    for match_data in turn_matches:
        match = {
            'match_id': match_data['partida_id'],
            'turn': turn,
            'home_id': match_data['clube_casa_id'],
            'visitor_id': match_data['clube_visitante_id'],
            'date': match_data['partida_data'],
            'timestamp': match_data['timestamp'],
            'local': match_data['local'],
            'valid': match_data['valida'],
            'home_goal': match_data['placar_oficial_mandante'],
            'visitor_goal': match_data['placar_oficial_visitante'],
            
        }
        matches.append(match)

In [ ]:
df_matches = pd.DataFrame(matches)
df_matches.shape

In [ ]:
df_matches.head(5)

### Manutenção do Banco de dados
Nessa seção do notebook os dados do banco da tabela match serão destruídos e os novos dados serão inseridos.

**Preencha os dados da conectividade:**

In [ ]:
user = 'postgres'
password = 'postgres'
address = '172.18.0.2'
db_name = 'cartola'

In [ ]:
def create_database_session():
    engine = create_engine(f'postgresql://{user}:{password}@{address}/{db_name}')
    return engine

In [ ]:
def erase_data(engine):
    engine.execute('TRUNCATE match')

In [ ]:
def create_table(engine):
    engine.execute('''
        CREATE TABLE IF NOT EXISTS match (
            match_id integer, 
            turn smallint,
            home_id smallint,
            visitor_id smallint,
            date varchar(20),
            timestamp bigint,
            local varchar(80),
            valid boolean,
            home_goal smallint,
            visitor_goal smallint
        )'''
    )

In [ ]:
def save_data(engine, dataframe):
    dataframe.to_sql('match', con=engine, index=False, if_exists='replace')

In [ ]:
def count_data(engine):
    return engine.execute('SELECT count(*) FROM match')

In [ ]:
engine = create_database_session()
create_table(engine)
erase_data(engine)
save_data(engine, df_matches)
count_data(engine).fetchall()

In [ ]:
end = timeit.default_timer()
print ('Duração: %.f segundo(os)' % ((end - start) ))